-------------------------------------------------------------------------------

CHIBO 2024

An attempt to static calculate the forces in (x,y,z) at all the suspension pickup points for various roll/jounce effects

18.12.2024: Created

03.01.2025: Outboard points calc working, moved functions to susprog.py and added loop 

05.01.2025: Added jounce updater for the inner points and included in susprog.py

08.01.2025: Added trail & scrub radius calc in loop

09.01.2025: Rewrote roll centre code to correct errors in original

11.01.2025: Added in trail/scrub/KPI/Caster angle evaluation. Bug was present for a while which miscalculated RC via unsmooth rotation of inner points. Went away for unknown reason

19.01.2025: Changed units to mm, this fixes small number rounding error which causes output instability

-------------------------------------------------------------------------------

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

import plotly.graph_objects as go
import matplotlib.pyplot as plt

import susprog                      # Python file containing the equations for the loop 

In [10]:
# Declare motions for evaluation

# Define maximum roll and jounce inputs
body_roll = -5.00  # Maximum roll angle in degrees
body_jounce = 0  # Maximum jounce in meters (5 cm = 0.05)
sim_vales = 100

# Generate roll and jounce motion inputs
roll_angles = np.linspace(0, np.radians(body_roll), sim_vales)  # 0 to max roll angle in 100 steps
jounce_displacements = np.linspace(0, body_jounce, sim_vales)  # 0 to max jounce in 100 steps

In [3]:
# Suspension pickup points in millimeters
upper_a_arm = np.array([
    [1.5, 506.5, 315],      # Outer point (converted from m to mm)
    [-135, 240, 220],       # Inner leading point
    [180, 255, 255]         # Inner trailing point
], dtype=np.float64)

lower_a_arm = np.array([
    [-17.5, 519, 137],      # Outer point
    [-180, 165, 87],        # Inner leading point
    [180, 170, 87]          # Inner trailing point
], dtype=np.float64)

tie_rod = np.array([
    [60, 515, 160],         # Outer point
    [70, 195, 100]          # Inner point
], dtype=np.float64)

tire_cop = np.array([-5, 525, 0], dtype=np.float64)      # Converted to mm
force_vector = np.array([50 * 1000, 500 * 1000, -2000 * 1000], dtype=np.float64)  # Force in N-mm


In [4]:
# Calculate lengths for upper and lower A-arms and tie rod
upper_lengths = [
    np.linalg.norm(upper_a_arm[1] - upper_a_arm[0]),  # Upper outer to inner leading
    np.linalg.norm(upper_a_arm[2] - upper_a_arm[0]),  # Upper outer to inner trailing
    np.linalg.norm(upper_a_arm[0] - lower_a_arm[0])   # Upper outer to lower outer
]

lower_lengths = [
    np.linalg.norm(lower_a_arm[1] - lower_a_arm[0]),  # Lower outer to inner leading
    np.linalg.norm(lower_a_arm[2] - lower_a_arm[0]),  # Lower outer to inner trailing
    np.linalg.norm(lower_a_arm[0] - tire_cop)         # Lower outer to tire center of pressure
]

tie_length = [
    np.linalg.norm(tie_rod[1] - tie_rod[0]),          # Tie rod outer to inner leading
    np.linalg.norm(upper_a_arm[0] - tie_rod[0]),      # Tie rod outer to upper outer
    np.linalg.norm(lower_a_arm[0] - tie_rod[0])       # Tie rod outer to lower outer
]

In [5]:
def get_plane_normal(points):
        """Calculate normal vector of plane defined by three points."""
        v1 = points[1] - points[0]
        v2 = points[2] - points[0]
        return np.cross(v1, v2)

In [10]:
def bump_steer_calc(outer_point, steering_axis_points, previous_angle=None):
    """
    Calculate the bump steer angle change for a given suspension iteration.
    
    Parameters:
        outer_point (np.array): Current position of the outer point (3D vector).
        steering_axis_points (np.array): Two points defining the steering axis (2x3 array).
        previous_angle (float, optional): The previous steering angle in radians. If None, only returns the current angle.

    Returns:
        float: The change in steering angle (if previous_angle is provided).
        float: The current steering angle in radians.
    """
    # Define the steering axis direction vector
    axis_point1 = steering_axis_points[0]
    axis_point2 = steering_axis_points[1]
    steering_axis = axis_point2 - axis_point1
    steering_axis /= np.linalg.norm(steering_axis)  # Normalize the vector

    # Vector from the axis to the outer point
    vector_to_outer = outer_point - axis_point1

    # Project the outer point vector onto the steering axis (normalized direction)
    projection = np.dot(vector_to_outer, steering_axis) * steering_axis

    # Perpendicular vector from the projection to the outer point
    perpendicular_vector = vector_to_outer - projection

    # Calculate the steering angle (angle between perpendicular vector and projection)
    angle = np.arctan2(
        np.linalg.norm(np.cross(steering_axis, vector_to_outer)),
        np.dot(steering_axis, vector_to_outer)
    )

    # If the previous angle is provided, calculate the change
    if previous_angle is not None:
        angle_change = angle - previous_angle
        return angle_change, angle

    return angle


In [11]:
# Run simulation

# Columns: [roll_angle (rad), jounce (m), suspension_positions (1x9), rc_x, rc_y, rc_z]
results = np.zeros((len(roll_angles), 30))  # 100 rows, 30 columns
#rc, geo_roll_centre = susprog.calculate_roll_center(upper_a_arm, lower_a_arm, tire_cop)
rc_data = susprog.calculate_rc_simple(upper_a_arm, lower_a_arm, tire_cop)
rc = rc_data[0:3]

# Set rotation point to origin
# rc = [0,0,0]

# Roll axis (example: x-axis)
roll_axis = np.array([1, 0, 0])
base_Steer = susprog.bump_steer_calc(upper_a_arm, lower_a_arm, tie_rod, None)

# Initialize rotated arms as a copy of the original
upper_modified = upper_a_arm.copy()
lower_modified = lower_a_arm.copy()
tierod_modified = tie_rod.copy()

# Loop through each roll and jounce input
for i, (roll_angle, jounce_disp) in enumerate(zip(roll_angles, jounce_displacements)):
    # Step 1: Rotate and jounce the inner points of the arms and tie rod
    upper_modified = susprog.rotate_points(upper_a_arm, rc, roll_axis, roll_angle)
    upper_modified = susprog.jounce_innerpoints(upper_modified, jounce_disp)

    lower_modified = susprog.rotate_points(lower_a_arm, rc, roll_axis, roll_angle)
    lower_modified = susprog.jounce_innerpoints(lower_modified, jounce_disp)
    
    tierod_modified = susprog.rotate_points(tie_rod, rc, roll_axis, roll_angle)
    tierod_modified = susprog.jounce_innerpoints(tierod_modified, jounce_disp)

    # Step 3: Update outer points of lower, upper, and tie rod
    # Lower A-arm
    points_temp = np.vstack((lower_modified, tire_cop))
    lower_modified[0:1] = susprog.solve_outer_position(points_temp, lower_lengths)
    del points_temp

    # Upper A-arm
    points_temp = np.vstack((upper_modified, lower_modified[0:]))
    upper_modified[0:1] = susprog.solve_outer_position(points_temp, upper_lengths)
    del points_temp

    # Tie rod
    points_temp = np.vstack((tierod_modified, upper_modified[0:1], lower_modified[0:1]))
    tierod_modified[0:1] = susprog.solve_outer_position(points_temp, tie_length)
    del points_temp


    # Step 4: Calculate the new roll center
    rc_data = susprog.calculate_rc_simple(upper_modified, lower_modified, tire_cop)
    rc = rc_data[0:3]
    
    # Caclulate the KPI/Caster, trail and scrub
    steering_data = susprog.suspension_geometry_calc(upper_modified, lower_modified, tire_cop)
    steering_data = [
        steering_data['trail'],  # trail in x
        steering_data['scrub'],  # scrub in y
        steering_data['kingpin_inclination_deg'],      # KPI
        steering_data['camber_angle_deg'],       # Caster
    ]
    
    # Calculate Bump Steer
    bump_data = susprog.bump_steer_calc(upper_modified, lower_modified, tierod_modified, base_Steer)
    

    # Step 5: Store the results
    results[i, 0] = roll_angle  # Roll angle (in radians)
    results[i, 1] = jounce_disp  # Jounce displacement
    results[i, 2:11] = lower_modified.flatten()  # Lower A-arm positions (3x3 -> 1x9)
    results[i, 11:20] = upper_modified.flatten()  # Upper A-arm positions (3x3 -> 1x9)
    results[i, 20:25] = rc_data  # Roll center (1x8)
    #results[i, 23] = geo_roll_centre['instant_center_2d'][0]  # IC x
    #results[i, 24] = geo_roll_centre['instant_center_2d'][1]  # IC z
    #results[i, 25] = geo_roll_centre['swing_arm_length']      # Swing arm length
    #results[i, 26] = geo_roll_centre['upper_arm_angle']       # Upper arm angle
    #results[i, 27] = geo_roll_centre['lower_arm_angle']       # Lower arm angle
    #results[i, 28:31] = geo_roll_centre['upper_normal']       # Upper plane normal (x, y, z)
    #results[i, 31:34] = geo_roll_centre['lower_normal']       # Lower plane normal (x, y, z)
    results[i, 25:29] = steering_data # Steering axis==> tire cop (1x3)
    results[i, 29] = bump_data

del i

# Convert results to a DataFrame for easier visualization and saving
column_names = [
    "roll_angle", "jounce", 
    "lower_outer_x", "lower_outer_y", "lower_outer_z", 
    "lower_inner1_x", "lower_inner1_y", "lower_inner1_z", 
    "lower_inner2_x", "lower_inner2_y", "lower_inner2_z",
    "upper_outer_x", "upper_outer_y", "upper_outer_z", 
    "upper_inner1_x", "upper_inner1_y", "upper_inner1_z", 
    "upper_inner2_x", "upper_inner2_y", "upper_inner2_z",
    "rc_x", "rc_y", "rc_z",
    "ic_y", "ic_z", 
    "Steering_trail_x", "Scrub_radius_y", "kingpin_inclination_deg", "camber_angle_deg",
    "Bump_Steer"
]
results_df = pd.DataFrame(results, columns=column_names)

# Save the results as a CSV file (optional)
results_df.to_csv("suspension_simulation_results.csv", index=False)

# Display the first few rows
#print(results_df.head())


In [12]:
# Extract the initial and final suspension points from the results
lower_initial = results[0, 2:11].reshape(3, 3)  # First row, lower A-arm (3x3)
upper_initial = results[0, 11:20].reshape(3, 3)  # First row, upper A-arm (3x3)

lower_final = results[-1, 2:11].reshape(3, 3)  # Last row, lower A-arm (3x3)
upper_final = results[-1, 11:20].reshape(3, 3)  # Last row, upper A-arm (3x3)

# Extract roll centers
rc_initial = results[0, 20:23]  # Initial roll center
rc_final = results[-1, 20:23]  # Final roll center

# Create a 3D scatter plot
fig = go.Figure()

# Add initial lower A-arm points
fig.add_trace(go.Scatter3d(
    x=lower_initial[:, 0], y=lower_initial[:, 1], z=lower_initial[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='blue'),
    name='Lower A-arm (Initial)'
))

# Add final lower A-arm points
fig.add_trace(go.Scatter3d(
    x=lower_final[:, 0], y=lower_final[:, 1], z=lower_final[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='red'),
    name='Lower A-arm (Final)'
))

# Add initial upper A-arm points
fig.add_trace(go.Scatter3d(
    x=upper_initial[:, 0], y=upper_initial[:, 1], z=upper_initial[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='green'),
    name='Upper A-arm (Initial)'
))

# Add final upper A-arm points
fig.add_trace(go.Scatter3d(
    x=upper_final[:, 0], y=upper_final[:, 1], z=upper_final[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='orange'),
    name='Upper A-arm (Final)'
))

# Add roll center points
fig.add_trace(go.Scatter3d(
    x=[rc_initial[0], rc_final[0]],
    y=[rc_initial[1], rc_final[1]],
    z=[rc_initial[2], rc_final[2]],
    mode='markers+lines',
    marker=dict(size=8, color='purple'),
    name='Roll Center'
))

# Configure the layout
fig.update_layout(
    scene=dict(
        xaxis_title='X-axis (m)',
        yaxis_title='Y-axis (m)',
        zaxis_title='Z-axis (m)'
    ),
    title="3D Suspension Geometry (Initial vs. Final)",
    legend=dict(x=0.1, y=0.9)
)

# Show the figure
fig.show()

# Print the initial and final points
print("Initial Lower A-arm Points:\n", lower_initial)
print("Final Lower A-arm Points:\n", lower_final)

print("Initial Upper A-arm Points:\n", upper_initial)
print("Final Upper A-arm Points:\n", upper_final)

print("Initial Roll Center:", rc_initial)
print("Final Roll Center:", rc_final)

Initial Lower A-arm Points:
 [[ -17.5  519.   137. ]
 [-180.   165.    87. ]
 [ 180.   170.    87. ]]
Final Lower A-arm Points:
 [[ -17.36887891  522.62378468  137.12272541]
 [-180.          171.09167191   72.3259207 ]
 [ 180.          176.0726454    71.89014199]]
Initial Upper A-arm Points:
 [[   1.5  506.5  315. ]
 [-135.   240.   220. ]
 [ 180.   255.   255. ]]
Final Upper A-arm Points:
 [[  -2.60250843  516.86700601  315.86841538]
 [-135.          257.39798805  198.28313484]
 [ 180.          275.39135952  231.84261313]]
Initial Roll Center: [ 0.          0.         10.61696163]
Final Roll Center: [0.         0.         9.91449626]


In [13]:
# Plot RC vs roll

fig = go.Figure()

# Add the trace for "rc_z"
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["rc_z"],
        mode='lines',
        marker=dict(color='orange'),
        name='Roll Centre Location, Z'
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Roll Centre Location',
    xaxis=dict(title='Roll Angle (radians)'),
    yaxis=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange')
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [16]:
# Plot RC vs jounce

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results_df["jounce"],
        y=results_df["rc_z"],
        mode='lines',
        marker=dict(color='red'),
        name='Roll Centre Location, z',
        yaxis='y'  # Explicitly associate this trace with the primary y-axis
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Roll Centre Location',
    xaxis=dict(title='Jounce (m)'),
    yaxis=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()


In [15]:
# Plot Bump steer vs roll

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["Bump_Steer"],
        mode='lines',
        marker=dict(color='red'),
        name='Roll Centre Location, z',
        yaxis='y'  # Explicitly associate this trace with the primary y-axis
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='bump',
    xaxis=dict(title='Jounce (m)'),
    yaxis=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()


In [39]:
# Plot inner positions vs roll

# Create the plot
fig = go.Figure()

# Add Z heights vs roll to the primary y-axis
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["lower_inner1_z"],
        mode='lines',
        marker=dict(color='red'),
        name='lower Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["upper_inner1_z"],
        mode='lines',
        marker=dict(color='orange'),
        name='Upper Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

# Add Y positions vs roll to the secondary y-axis
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["lower_inner1_y"],
        mode='lines',
        marker=dict(color='blue'),
        name='Lower Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["upper_inner1_y"],
        mode='lines',
        marker=dict(color='green'),
        name='Upper Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)

# Update layout to add the secondary y-axis
fig.update_layout(
    title='Inner Leading Positions vs Roll',
    xaxis=dict(title='Roll Angles (degrees)'),
    yaxis=dict(
        title='Z Heights (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='Y Positions (m)',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [14]:
# Plot KPI/Caster vs roll

# Create the plot
fig = go.Figure()

# Add king pin incl
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["kingpin_inclination_deg"],
        mode='lines',
        marker=dict(color='red'),
        name='KPI (°)',
        yaxis='y'  # Primary y-axis
    )
)

# Add caster angle 
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["camber_angle_deg"],
        mode='lines',
        marker=dict(color='blue'),
        name='Caster Angle (°)',
        yaxis='y2'  # Secondary y-axis
    )
)

# Update layout to add the secondary y-axis
fig.update_layout(
    title='Inner Leading Positions vs Roll',
    xaxis=dict(title='Roll Angles (degrees)'),
    yaxis=dict(
        title='KPI [°]',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='Caster angle [°]',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [35]:
# Plot KPI/Caster vs jounce

# Create the plot
fig = go.Figure()

# Add king pin incl
fig.add_trace(
    go.Scatter(
        x=results_df["jounce"],
        y=results_df["kingpin_inclination_deg"],
        mode='lines',
        marker=dict(color='red'),
        name='lower Leading Z Heights',
        yaxis='y'  # Primary y-axis
    )
)

# Add caster angle 
fig.add_trace(
    go.Scatter(
        x=results_df["jounce"],
        y=results_df["camber_angle_deg"],
        mode='lines',
        marker=dict(color='blue'),
        name='Lower Leading Y Positions',
        yaxis='y2'  # Secondary y-axis
    )
)

# Update layout to add the secondary y-axis
fig.update_layout(
    title='Inner Leading Positions vs Roll',
    xaxis=dict(title='body jounce (m)'),
    yaxis=dict(
        title='KPI [°]',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='Caster angle [°]',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()